In [ ]:
#  필요한 라이브러리 불러오기
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib.pyplot as plt 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
    
import re
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots    
    
    

* What is the picture of digital connectivity and engagement in 2020?  
#product_df.csv & lp_id ---- engagement_df.csv & lp_id   
 
* What is the effect of the COVID-19 pandemic on online and distance learning, and how might this also evolve in the future?  
#어떤 product(zoom, ~ ) 를 많이 썼느냐   
  
* How does student engagement with different types of education technology change over the course of the pandemic?  
#366일을 데이터를 다 가지고 있는 district와 366일 미만의 데이터를 가지고 있는 district 구별  


* How does student engagement with online learning platforms relate to different geography? Demographic context (e.g., race/ethnicity, ESL, learning disability)? Learning context? Socioeconomic status?


* Do certain state interventions, practices or policies (e.g., stimulus, reopening, eviction moratorium) correlate with the increase or decrease online engagement?

In [ ]:
path = os.getcwd()
print(path)

In [ ]:
products_df = pd.read_csv("/kaggle/input/learnplatform-covid19-impact-on-digital-learning/products_info.csv")
districts_df = pd.read_csv("/kaggle/input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv")

districts_df.shape, products_df.shape


In [ ]:
# engagement_data 폴더의 파일 확인
os.listdir("../input/learnplatform-covid19-impact-on-digital-learning/engagement_data")

In [ ]:
districts_df.head()

In [ ]:
products_df.head()

In [ ]:
# engagement_data 폴더의 파일 확인
engagement_df= os.listdir("../input/learnplatform-covid19-impact-on-digital-learning/engagement_data")
engagement_df[0:10]

In [ ]:
engagement_1000_df = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/engagement_data/1000.csv")
engagement_1000_df.head()

### Preprocessing
* dropping 57 school districts with NaN states (57/233 ~ 25%)
* one-hot encoding the product sectors
* splitting up the primary essential function into main and sub category

In [ ]:
districts_df.isna().sum()

In [ ]:
# 전체 state 값 
print(districts_df.shape)
districts_df = districts_df[districts_df.state.notna()].reset_index(drop=True)
# state 의 NAN을 뺀다.
print(districts_df.shape)

In [ ]:
products_df['Sector(s)'].unique()

In [ ]:
temp_sectors = products_df['Sector(s)'].str.get_dummies(sep="; ")   # 문자열 값 중에,  세개의 변수를 만들겠다.
temp_sectors.head()

In [ ]:
temp_sectors.columns = [f"sector_{re.sub(' ', '', c)}" for c in temp_sectors.columns]  # 공백을 없애고, 

In [ ]:
temp_sectors.columns

In [ ]:
products_df = products_df.join(temp_sectors)
products_df.head()

In [ ]:
products_df.drop("Sector(s)", axis=1, inplace=True)
print(products_df.columns)
del temp_sectors

### Primary Essential Function의 LC - Digital~~ 나누기

In [ ]:
products_df['pri_function_main'] = products_df['Primary Essential Function'].apply(lambda x: x.split(' - ')[0] if x == x else x)
products_df['pri_function_sub'] = products_df['Primary Essential Function'].apply(lambda x: x.split(' - ')[1] if x == x else x)

In [ ]:
products_df.head()

In [ ]:
# Synchronize similar values
products_df['pri_function_sub'] = products_df['pri_function_sub'].replace(
          {'Sites, Resources & References' : 'Sites, Resources & Reference'})
products_df.drop("Primary Essential Function", axis=1, inplace=True)

In [ ]:
products_df['pri_function_sub'].unique()

In [ ]:
products_df[ ['sector_Corporate', 'sector_HigherEd', 'sector_PreK-12',
       'pri_function_main', 'pri_function_sub'] ]

### engagement_data의 데이터 파일에 district_id를 추가

In [ ]:
districts_df.district_id.unique()

In [ ]:
PATH = "../input/learnplatform-covid19-impact-on-digital-learning/engagement_data"

temp = []

for district in districts_df.district_id.unique():
    df = pd.read_csv(f'{PATH}/{district}.csv', index_col=None, header=0)
    df['district_id'] = district
    temp.append(df)

len(temp)

In [ ]:

engagement = pd.concat(temp)
engagement = engagement.reset_index(drop=True)
engagement.head()


In [ ]:
engagement.tail()

* 대부분의 학군에는 366일의 고유 일수가 있습니다.
* 그러나 43개의 학군의 경우, 366일 미만의 고유한 데이터를 사용할 수 있음.
* district_id 3670의 경우, 2020-02-15부터 2020-03-02의 데이터만 사용이 가능.
* district_id 2872의 경우, 2020년 1월의 데이터만 사용 가능하고, 2월과 3월의 각각 1일, 총 2일만 데이터 사용가능

In [ ]:
len(engagement.district_id.unique())

In [ ]:

engagement.info()

In [ ]:
engagement[engagement['district_id']==3670].time.unique()

### 각 district_id에 대해 사용가능한 날짜 카운트를 히스토그램으로 표현

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,4))

sns.histplot(engagement.groupby('district_id').time.nunique(), bins=30)
ax.set_title('Unique Days of Engagement Data per District')
plt.show()

In [ ]:
# 앞에서 확인한 engagement를 지우고, 새롭게 만든다.
del engagement

temp = []

for district in districts_df.district_id.unique():
    df = pd.read_csv(f'{PATH}/{district}.csv', index_col=None, header=0)
    df["district_id"] = district
    if df.time.nunique() == 366:    # 366일인 것만 추가하겠다. 
        temp.append(df)

engagement = pd.concat(temp)   # 366일인것만 뽑아서 pandas로 찍기 
engagement = engagement.reset_index(drop=True)

In [ ]:
districts_df.shape, products_df.shape

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,4))

sns.histplot(engagement.groupby('district_id').time.nunique(), bins=30)
ax.set_title('Unique Days of Engagement Data per District')
plt.show()

### 전체 2020년이 있는 데이터만 합친다.

* 컬럼명과 파일명 연결시키기

In [ ]:
districts_df = districts_df[districts_df.district_id.isin(engagement.district_id.unique())].reset_index(drop=True)
products_df = products_df[products_df['LP ID'].isin(engagement.lp_id.unique())].reset_index(drop=True)

In [ ]:
districts_df.shape, products_df.shape

In [ ]:
engagement.time = engagement.time.astype('datetime64[ns]')

### EDA(Exploratory Data Analysis)
* 이용 가능한 학군 확인.
* 학군이 가장 많이 있는 주는 CT(29)와 UT(24)
* 학군이 하나만 있는 주는 (FL, TN, NY, AZ)

In [ ]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District Of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [ ]:
districts_df['state_abbrev'] = districts_df['state'].replace(us_state_abbrev)
districts_df_by_state = districts_df['state_abbrev'].value_counts().to_frame().reset_index(drop=False)
districts_df_by_state.head()

In [ ]:
districts_df_by_state.columns = ['state_abbrev', 'num_districts']

fig = go.Figure()
layout = dict(
    title_text = "Number of Available School Districts per State",
    geo_scope='usa',
)

fig.add_trace(
    go.Choropleth(
        locations=districts_df_by_state.state_abbrev,
        zmax=1,
        z = districts_df_by_state.num_districts,
        locationmode = 'USA-states', # set of locations match entries in `locations`
        marker_line_color='white',
        geo='geo',
        colorscale=px.colors.sequential.Teal, 
    )
)
            
fig.update_layout(layout)   
fig.show()

### 'Primary Essential Function'의 열에서 가장 일반적인 범주는 LC(learning & curriculum), 교실 관리(CM)및 학교 및 학군 운영(SDO)

In [ ]:
#countplot으로 개수 찍어보기

fig, ax = plt.subplots(1, 2, figsize=(16,4))
sns.countplot(data=products_df, x='pri_function_main', palette ='GnBu', ax=ax[0])
ax[0].set_title('Main Categories in Primary Functions')

sns.countplot(data=products_df[products_df.pri_function_main == 'LC'], 
                       x='pri_function_sub', palette ='GnBu', ax=ax[1])

ax[1].set_title('Sub-Categories in Primary Function LC')
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=90)
plt.show()

In [ ]:
virtual_classroom_lp_id = products_df[
                          products_df.pri_function_sub == 'Virtual Classroom']['LP ID'].unique()

# Remove weekends from the dataframe
engagement['weekday'] = pd.DatetimeIndex(engagement['time']).weekday
engagement_without_weekends = engagement[engagement.weekday < 5]

# Figure 1
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(24, 6))
for virtual_classroom_product in virtual_classroom_lp_id:
    temp = engagement_without_weekends[
            engagement_without_weekends.lp_id == virtual_classroom_product
        ].groupby('time').pct_access.mean().to_frame().reset_index(drop=False)
    sns.lineplot(x=temp.time, y=temp.pct_access, 
                 label=products_df[
                 products_df['LP ID'] == virtual_classroom_product]['Product Name'].values[0])
plt.legend()
plt.show()

# Figure 2
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(24, 6))
for virtual_classroom_product in virtual_classroom_lp_id:
    temp = engagement_without_weekends[
            engagement_without_weekends.lp_id == virtual_classroom_product
           ].groupby('time').engagement_index.mean().to_frame().reset_index(drop=False)
    sns.lineplot(x=temp.time, 
                 y=temp.engagement_index, 
                 label=products_df[
                     products_df['LP ID'] == virtual_classroom_product]['Product Name'].values[0])
plt.legend()
plt.show()

### 확인된 내용
* 홈 스쿨링은 3월 초에 시작
* 3월과 7월 사이에 종모양이 있음.
* 7월과 8월에는 여름 방학이 있으므로 참석할 수업이 없음.
* 여름 방학 이후 pct_access는 전염병 초기에 관찰된 대로 더 높은 수준으로 증가 다소 일정하게 유지
* 연중 내내 pct_access에 몇몇 하락이 모인다. 이는 공휴일 또는 기타 공휴일 가능성이 있음.
* Zoom과 Meet는 가상 교실에서 가장 인기 있는 제품.

In [ ]:
products_df.head()

In [ ]:
display(products_df.sum())

display(products_df.groupby('pri_function_main')['pri_function_sub'].value_counts().to_frame())

### Summary
* 달성하고자 하는 바에 따라 신중하게 구역을 사전 선택하는 것이 좋다.
* 노트북에서 접근하는 방식이 개인의 목적에 맞지 않을 수도 있음.
* 디지털 학습을 볼 때 실제로 디지털 학습을 적용한 학군을 파악하는데 시간을 할애할 수 있음.